#GP-regression

Рассмотрим задачу GP-регрессии с обучающей выборкой $X \in \mathbb{R}^{d \times n}$, $y \in \mathbb{R}^n$. 

Будем обозначать через $X_*$ тестовые примеры, а через $f_*$ — случайный вектор, определяемый гауссовским процессом в тестовых точках. Определим априорное распределение $f_* \sim \mathcal{N}(0, K(X_*, X_*))$, где $K(\cdot, \cdot)$ — ковариационная функция. Тогда априорное совместное распределение
$$ 
\left[
\begin{matrix}
f\\
f_*
\end{matrix}
\right ] 
\sim
\mathcal{N}
\left( 0,
\left[
\begin{matrix}
K(X, X) & K(X, X_*)\\
K(X_*, X) & K(X_*, X_*)
\end{matrix}
\right]
\right).
$$


Будем считать, что наблюдаемые значения являются зашумленной реализацией гауссовского процесса $y = f(x) + \varepsilon$, где $\varepsilon \sim \mathcal{N}(0, \sigma_n^2)$, $f(x) \sim \mathcal{GP}(m(x), k(x, x'))$. Тогда 
$$cov(y_p, y_q) = K(x_p, x_q) + \delta_{pq} \sigma_n^2(p, q).$$

Тогда совместное распределение наблюдаемых значений $y$ в точках $X$ и неизвестных значений $f_*$ в точках $X_*$
$$ 
\left[
\begin{matrix}
y\\
f_*
\end{matrix}
\right ] 
\sim
\mathcal{N}
\left( 0,
\left[
\begin{matrix}
K(X, X) + \sigma_n^2 I & K(X, X_*)\\
K(X_*, X) & K(X_*, X_*)
\end{matrix}
\right]
\right).
$$

Можем выписать условное распределение 
$$f_* \rvert y, X, X_* \sim \mathcal{N}(\bar f_*, cov(f_*)), $$
где 
$$\bar f_* = K(X_*, X)[K(X, X) + \sigma_n^2 I]^{-1} y, $$
$$ cov(f_*) = K(X_*, X_*) - K(X_*, X) [K(X, X) + \sigma_n^2 I]^{-1} K(X, X^*).$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Covariance functions

Будем считать, что $X, Y$ — матрицы размера $d \times n$, где $d$ размерность, а $n$ количество точек. Ковариационная функция cov(X, Y)возвращает матрицу $K(X, Y)$ такую что $K(X, Y)_{ij} = cov(X^i, Y^j)$.

In [2]:
def covariance_mat(covariance_func, X, Y):
    """Computing covariance matrix for given covariance function and point arrays"""
    mat = np.zeros((X.shape[1], Y.shape[1]))
    for i in range(0, X.shape[1]):
        for j in range(0, Y.shape[1]):
            mat[i, j] = covariance_func(X[:, i], Y[:, j])
    return mat

In [3]:
def delta(x, y):
    return max(x == y)

Squared exponential covariance

In [4]:
def squared_exponential_cov(alpha, sigma, l):
    """Squared exponential (SE) covariance function"""
    def se_cov(x, y):
        return np.exp(-np.square(np.linalg.norm(x - y) / (2*l**2))) * alpha + sigma**2 * delta(x, y)
    return se_cov

## Sampling

In [1]:
def sample(mean_func, cov_func, x):
    cov_mat = covariance_mat(cov_func, x, x)
    m_v = mean_func(x)
    mean_vector = m_v.reshape((m_v.size,))
    y = np.random.multivariate_normal(mean_vector, cov_mat)
    return (y)

In [15]:
def sample_at_points(mean_vec, cov_mat):
    y = np.random.multivariate_normal(mean_vec.reshape((mean_vec.size,)), cov_mat)
    upper_bound = mean_vec + 3 * np.sqrt(np.diagonal(cov_mat).reshape(mean_vec.shape))
    lower_bound = mean_vec - 3 * np.sqrt(np.diagonal(cov_mat).reshape(mean_vec.shape))
    return (y, upper_bound, lower_bound)

##Visualization

In [16]:
def plotdata(x, y, color):
    x1 = x.reshape((x.size, ))
    y1 = y.reshape((y.size, ))
    plt.plot(x1, y1, color)

In [17]:
def draw_sample(mean_func, cov_func, x_0, x_1):
    plt.axis([x_0, x_1, -2, 2])
    x = np.linspace(x_0, x_1, 100)
    x = x.reshape((1, x.size))
    cov_mat = covariance_mat(K, x, x)
    m_v = m(x)
    mean_vector = m_v.reshape((m_v.size,))
    upper_bound = m_v + 3 * np.sqrt(np.diagonal(cov_mat))
    lower_bound = m_v - 3 * np.sqrt(np.diagonal(cov_mat))
#     y = np.random.multivariate_normal(mean_vector, cov_mat)
    y = sample(mean_func, cov_func, x)
    plotdata(x.reshape((x.size, )), y, 'b')
    plotdata(x.reshape((x.size, )), upper_bound.reshape((x.size, )), 'r')
    plotdata(x.reshape((x.size, )), lower_bound.reshape((x.size, )), 'g')

##Parameters

Common

In [18]:
x_0, x_1 = -10, 10
density = 100

Data generation

In [19]:
alpha_g, sigma_g, l_g = 1.0, 0.1, 1.0# 
n = 20 # number of examples

Prior distribution

In [20]:
alpha, sigma, l = 1.0, 0.1, 1.0

##Data generation

In [21]:
def mean_val(x):
    return 0
m = np.vectorize(mean_val)

In [22]:
K_g = squared_exponential_cov(alpha_g, sigma_g, l_g)
x_g = x_0 + np.random.rand(1, n)*(x_1 - x_0)
y_g = sample(m, K_g, x_g)
y_g = y_g.reshape((y_g.size, 1))

In [23]:
plotdata(x_g, y_g, 'x')
plt.show()

##Computations

Initialazing the covariance and mean

In [24]:
K = squared_exponential_cov(alpha, sigma, l)
def mean_val(x):
    return 0
m = np.vectorize(mean_val)

In [25]:
draw_sample(m, K, x_0, x_1)
plotdata(x_g, y_g, 'x')
plt.show()

In [26]:
x_test = np.linspace(x_0, x_1, density).reshape((1, density))
K_x = covariance_mat(K, x_g, x_g)
K_x_test = covariance_mat(K, x_g, x_test)
K_test_x = covariance_mat(K, x_test, x_g)
K_test = covariance_mat(K, x_test, x_test)
I = np.eye(K_x.shape[0])

In [27]:
anc_mat = np.linalg.inv(K_x + I * sigma**2)

In [28]:
new_mean = np.dot(np.dot(K_test_x, anc_mat), y_g)

In [29]:
new_cov = K_test - np.dot(np.dot(K_test_x, anc_mat), K_x_test)

In [30]:
y_test, u_b, l_b = sample_at_points(new_mean, new_cov)

In [ ]:
plotdata(x_g, y_g, 'x')
plotdata(x_test, u_b, 'r')
plotdata(x_test, l_b, 'g')
plotdata(x_test, y_test, 'b')
plt.show()